In [2]:
import numpy as np
import time
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer
model_path = "/dataset/crosspipe/OriginModel/Llama-2-7b-chat-hf/"
device = "cuda:2" if torch.cuda.is_available() else "cpu"
model = LlamaForCausalLM.from_pretrained(model_path, device_map=device)
tokenizer = LlamaTokenizer.from_pretrained(model_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [1]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer

# 加载模型和分词器
model_path = "/dataset/crosspipe/OriginModel/Llama-2-7b-chat-hf/"
device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = LlamaForCausalLM.from_pretrained(model_path).to(device)
tokenizer = LlamaTokenizer.from_pretrained(model_path)

# 输入文本
input_text = "Once upon a time"
inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)

# 获取模型的输出，包括 hidden states
with torch.no_grad():
    outputs = model(
        inputs,
        output_hidden_states=True,  # 启用 hidden states 的输出
        return_dict=True  # 返回字典形式的输出，便于访问
    )

# 获取 hidden states
hidden_states = outputs.hidden_states

# 查看每一层的 hidden states 及其维度
for i, layer_hidden_states in enumerate(hidden_states):
    print(f"Layer {i} hidden states shape: {layer_hidden_states.shape}")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


Layer 0 hidden states shape: torch.Size([1, 5, 4096])
Layer 1 hidden states shape: torch.Size([1, 5, 4096])
Layer 2 hidden states shape: torch.Size([1, 5, 4096])
Layer 3 hidden states shape: torch.Size([1, 5, 4096])
Layer 4 hidden states shape: torch.Size([1, 5, 4096])
Layer 5 hidden states shape: torch.Size([1, 5, 4096])
Layer 6 hidden states shape: torch.Size([1, 5, 4096])
Layer 7 hidden states shape: torch.Size([1, 5, 4096])
Layer 8 hidden states shape: torch.Size([1, 5, 4096])
Layer 9 hidden states shape: torch.Size([1, 5, 4096])
Layer 10 hidden states shape: torch.Size([1, 5, 4096])
Layer 11 hidden states shape: torch.Size([1, 5, 4096])
Layer 12 hidden states shape: torch.Size([1, 5, 4096])
Layer 13 hidden states shape: torch.Size([1, 5, 4096])
Layer 14 hidden states shape: torch.Size([1, 5, 4096])
Layer 15 hidden states shape: torch.Size([1, 5, 4096])
Layer 16 hidden states shape: torch.Size([1, 5, 4096])
Layer 17 hidden states shape: torch.Size([1, 5, 4096])
Layer 18 hidden stat

: 

In [4]:
context_ids = inputs[:, :-1]
next_word_ids = inputs[:, -1:]

In [5]:
inputs
context_ids
next_word_ids

tensor([[931]], device='cuda:2')

In [6]:
attention_weights = outputs.attentions

In [9]:
if outputs.attentions is not None:
    for layer_idx, layer_attention in enumerate(outputs.attentions):
        print(f"Layer {layer_idx}: {layer_attention[-1].shape}")
else:
    print("Attention weights are None!")


Attention weights are None!


In [2]:
outputs

GenerateDecoderOnlyOutput(sequences=tensor([[    1,   525, 26222,  2501,   263,   931, 29915,   297,   278,  3186,
           310,  1436,   749,    13,    13, 26222,  2501,   263,   931, 29892,
           297,   263,  2982,  2215, 29892,  2215,  3448, 29892,   727,   471,
           263,  2320,   936, 20748,  2000,  4231,   749,  4909, 29889,   512,
           445,   427,   305,  9714,  2982, 29892,   278, 19681,   892,  2998,
           408,  4231,  8463,   550, 29892,   322]], device='cuda:2'), scores=(tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:2'), tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:2'), tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:2'), tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:2'), tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:2'), tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cuda:2'), tensor([[-inf, -inf, -inf,  ..., -inf, -inf, -inf]], device='cud

In [7]:
generated_ids = outputs.sequences
generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
print(generated_text)

["'Once upon a time' is a phrase that is often used to introduce a story, but it can also be used to describe a time in the past that is remembered or imagined. Here are some examples of how 'once upon a time' can be used in different"]


In [19]:
past_key_values = outputs.past_key_values

In [21]:
print(type(past_key_values))

<class 'tuple'>


In [23]:
print(len(past_key_values))

32


In [28]:
past_key_value1 = (
    torch.tensor([[[[1, 2], [3, 4]], [[5, 6], [7, 8]]], [[[9, 10], [11, 12]], [[13, 14], [15, 16]]]]),
    torch.tensor([[[[17, 18], [19, 20]], [[21, 22], [23, 24]]], [[[25, 26], [27, 28]], [[29, 30], [31, 32]]]])
)

In [34]:
for key_states, value_states in past_key_value1:
    
    print(key_states)

tensor([[[1, 2],
         [3, 4]],

        [[5, 6],
         [7, 8]]])
tensor([[[17, 18],
         [19, 20]],

        [[21, 22],
         [23, 24]]])


In [26]:
for past_key, past_value in past_key_values:
    print(f"Key shape: {past_key.shape}")
    print(f"Value shape: {past_value.shape}")

Key shape: torch.Size([1, 32, 55, 128])
Value shape: torch.Size([1, 32, 55, 128])
Key shape: torch.Size([1, 32, 55, 128])
Value shape: torch.Size([1, 32, 55, 128])
Key shape: torch.Size([1, 32, 55, 128])
Value shape: torch.Size([1, 32, 55, 128])
Key shape: torch.Size([1, 32, 55, 128])
Value shape: torch.Size([1, 32, 55, 128])
Key shape: torch.Size([1, 32, 55, 128])
Value shape: torch.Size([1, 32, 55, 128])
Key shape: torch.Size([1, 32, 55, 128])
Value shape: torch.Size([1, 32, 55, 128])
Key shape: torch.Size([1, 32, 55, 128])
Value shape: torch.Size([1, 32, 55, 128])
Key shape: torch.Size([1, 32, 55, 128])
Value shape: torch.Size([1, 32, 55, 128])
Key shape: torch.Size([1, 32, 55, 128])
Value shape: torch.Size([1, 32, 55, 128])
Key shape: torch.Size([1, 32, 55, 128])
Value shape: torch.Size([1, 32, 55, 128])
Key shape: torch.Size([1, 32, 55, 128])
Value shape: torch.Size([1, 32, 55, 128])
Key shape: torch.Size([1, 32, 55, 128])
Value shape: torch.Size([1, 32, 55, 128])
Key shape: torch

In [10]:
past_key_values

((tensor([[[[-4.4985e-01,  5.7186e-03,  6.7457e-02,  ...,  1.1431e-01,
             -7.8889e-02, -5.8938e-02],
            [ 6.7588e-02, -4.4601e-01, -5.3787e-03,  ...,  8.8589e-01,
             -8.3431e-01,  7.9908e-01],
            [-2.2422e-01,  4.6389e-02,  9.0266e-02,  ...,  3.0349e-01,
             -5.5649e-02,  3.9021e-01],
            ...,
            [ 4.5479e-01, -5.9381e-01, -2.1023e-01,  ..., -2.9766e-01,
              3.5301e-01, -2.5323e-01],
            [-3.1056e-02,  8.6566e-03, -1.9548e-01,  ...,  3.1935e-01,
             -1.8813e-01,  2.1785e-01],
            [-5.3031e-01,  5.0688e-01, -9.8389e-01,  ..., -3.9444e-01,
              3.3828e-01, -4.1323e-01]],
  
           [[ 1.3652e+00,  1.0850e+00, -4.1402e-01,  ...,  5.7588e-01,
             -2.7642e-01,  5.5840e-01],
            [-1.0035e-01,  1.7187e-01,  1.9415e-01,  ..., -8.4641e-01,
              4.6206e-01, -6.8935e-01],
            [-1.9977e-01,  2.1759e-01,  1.1093e-01,  ...,  1.3152e-01,
              1.4925

In [17]:
past_key_values

((tensor([[[[-4.4985e-01,  5.7186e-03,  6.7457e-02,  ...,  1.1431e-01,
             -7.8889e-02, -5.8938e-02],
            [ 6.7588e-02, -4.4601e-01, -5.3787e-03,  ...,  8.8589e-01,
             -8.3431e-01,  7.9908e-01],
            [-2.2422e-01,  4.6389e-02,  9.0266e-02,  ...,  3.0349e-01,
             -5.5649e-02,  3.9021e-01],
            ...,
            [ 4.5479e-01, -5.9381e-01, -2.1023e-01,  ..., -2.9766e-01,
              3.5301e-01, -2.5323e-01],
            [-3.1056e-02,  8.6566e-03, -1.9548e-01,  ...,  3.1935e-01,
             -1.8813e-01,  2.1785e-01],
            [-5.3031e-01,  5.0688e-01, -9.8389e-01,  ..., -3.9444e-01,
              3.3828e-01, -4.1323e-01]],
  
           [[ 1.3652e+00,  1.0850e+00, -4.1402e-01,  ...,  5.7588e-01,
             -2.7642e-01,  5.5840e-01],
            [-1.0035e-01,  1.7187e-01,  1.9415e-01,  ..., -8.4641e-01,
              4.6206e-01, -6.8935e-01],
            [-1.9977e-01,  2.1759e-01,  1.1093e-01,  ...,  1.3152e-01,
              1.4925

In [116]:
output = model.generate(input_ids=next_word_ids, past_key_values=past_key_values)

RuntimeError: upper bound and larger bound inconsistent with step sign

In [ ]:
# 继续生成文本
input_text_continued = "And they come across a huge monster"
inputs_continued = tokenizer(input_text_continued, return_tensors="pt").to(device)
inputs_continued

In [ ]:
# 继续生成文本并传递以前的 KV 缓存
outputs_continued = model.generate(
    **inputs_continued,
    max_new_tokens=50,
    use_cache=True,
    return_dict_in_generate=True,
    output_scores=True,
    past_key_values=past_key_values  # 传递以前的 KV 缓存
)

# 解码生成的文本
generated_text = tokenizer.decode(outputs_continued.sequences[0], skip_special_tokens=True)
print(generated_text)

In [2]:
input_text = "'Once upon a time"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

In [7]:
input_ids

tensor([[    1,   525, 26222,  2501,   263,   931]], device='cuda:3')

In [3]:
out = model.generate(input_ids,  max_new_tokens=50, use_cache=True, return_dict_in_generate=True)
past_key_values = out.past_key_values
generated_ids = out.sequences

# Now we can continue generation using cache and already generated tokens
out_continued = model.generate(generated_ids,  max_new_tokens=50,past_key_values=past_key_values, return_dict_in_generate=True)
continued_generated_ids = out_continued.sequences

In [4]:
generated_output = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
continued_output = tokenizer.batch_decode(continued_generated_ids, skip_special_tokens=True)

In [5]:
generated_output

["'Once upon a time, in a land far, far away...' is a classic opening line used to transport readers to a magical world of wonder and enchantment. But what if the story takes place in a world that is not so far away? What if"]

In [6]:
continued_output

["'Once upon a time, in a land far, far away...' is a classic opening line used to transport readers to a magical world of wonder and enchantment. But what if the story takes place in a world that is not so far away? What if the magic and wonder are hidden in plain sight, waiting to be discovered?\n\nIn this collection of short stories, we explore the idea of magic and wonder in the everyday world. From the secret garden hidden behind a dumpster to the mag"]